In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import roc_auc_score,accuracy_score,r2_score
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [6]:
train = pd.read_csv('train.csv').set_index('id')

In [7]:
train.isnull().sum()

allelectrons_Total       0
density_Total            0
allelectrons_Average     0
val_e_Average            0
atomicweight_Average     0
ionenergy_Average        0
el_neg_chi_Average       0
R_vdw_element_Average    0
R_cov_element_Average    0
zaratio_Average          0
density_Average          0
Hardness                 0
dtype: int64

In [8]:
train.head(2)

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.86,0.496070,0.91457,6.0
1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.91,0.492719,0.71760,6.5


In [11]:
X= train.drop('Hardness', axis=1)
y=train['Hardness']

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10407 entries, 0 to 10406
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   allelectrons_Total     10407 non-null  float64
 1   density_Total          10407 non-null  float64
 2   allelectrons_Average   10407 non-null  float64
 3   val_e_Average          10407 non-null  float64
 4   atomicweight_Average   10407 non-null  float64
 5   ionenergy_Average      10407 non-null  float64
 6   el_neg_chi_Average     10407 non-null  float64
 7   R_vdw_element_Average  10407 non-null  float64
 8   R_cov_element_Average  10407 non-null  float64
 9   zaratio_Average        10407 non-null  float64
 10  density_Average        10407 non-null  float64
 11  Hardness               10407 non-null  float64
dtypes: float64(12)
memory usage: 1.0 MB


In [13]:
dtr = DecisionTreeRegressor(random_state=24)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
params = {'min_samples_split':[2,10,25,50], 'min_samples_leaf':[1,10,25,50],'max_depth':[None,4,6]}
gcv = GridSearchCV(dtr,param_grid=params,cv=kfold,scoring='r2',verbose=3)
gcv.fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.052 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.117 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.058 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.080 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.086 total time=   0.1s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.091 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.049 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.065 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.069 total time=   0.0s
[CV 5/5] EN

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=24),
             param_grid={'max_depth': [None, 4, 6],
                         'min_samples_leaf': [1, 10, 25, 50],
                         'min_samples_split': [2, 10, 25, 50]},
             scoring='r2', verbose=3)

In [14]:
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': None, 'min_samples_leaf': 50, 'min_samples_split': 2}
0.39799384830214196


In [15]:
best_tree = gcv.best_estimator_
best_tree

DecisionTreeRegressor(min_samples_leaf=50, random_state=24)

In [18]:
test= pd.read_csv("test.csv").set_index('id')
test.head(2)

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
id,,,,,,,,,,,
10407,884.0,121.42000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200,0.461899,1.79459
10408,90.0,9.93196,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864,0.476196,1.41194


In [19]:
y_pred= best_tree.predict(test)

In [20]:
submit  = pd.read_csv('sample_submission.csv')

In [21]:
submit

,id,Hardness
0,10407,4.647
1,10408,4.647
2,10409,4.647
3,10410,4.647
4,10411,4.647
...,...,...
6934,17341,4.647
6935,17342,4.647
6936,17343,4.647
6937,17344,4.647


In [22]:
submit['Hardness']= y_pred

In [23]:
submit

,id,Hardness
0,10407,2.772093
1,10408,3.201370
2,10409,5.632184
3,10410,4.292045
4,10411,4.969231
...,...,...
6934,17341,6.231373
6935,17342,3.430986
6936,17343,5.632184
6937,17344,4.150794


In [24]:
submit.to_csv('ss_8_nov_1_lr.csv',index=False)